In [4]:
#av_expon (expon for antithetic variates)

#This is to show how seed(), getstate() and setstate() can be used to
#manipulate a random number generator so that it gives you the same
#uniform number in cases where you need to use the complement of this
#number for generating correlated variates from some distribution. Here
#we use it for the exponential distribution.

#You may arrange things in a different way in your code. This is simply to
#explain the idea. Be sure that you understand exactly what is going on
#because it is easy to end up using the wrong random numbers.

#Another way is go about all this is to save the uniform random numbers you
#use in the first run of each pair in a list and then get the complement
#at run time for the second run of each pair. This will also work but is
#cumbersome as a general method.

import sys
import random as ran
from math import log as ln

class Expon:
    
    def __init__(self, xmean):  #initialize the object
        self.low = 0.00000001
        self.high = 0.99999999
        self.mean = xmean

    def setseed(self, seed):  #set the seed
        ran.seed(seed)

    def restore_seed(self): #rng has a "state" which can be more than the seed
        ran.setstate(self.state) #restore the previously stored state so we can
                                 #get the same uniform random number again and
                                 #compute its complement 

    def unif(self):  #get a U(0,1) random number, but save the rng state first
        self.state = ran.getstate()
        u = ran.uniform(self.low,self.high)
        return(u)
    
    def c_unif(self): #get the complement of a U(0,1) number. If you restore
                      #the seed before calling unif() the second time, you'll
                      #get the same value of u, and thus the correct (1-u)
        u = self.unif()
        return(1-u)#

    def expon(self): #xmean is the mean and not the rate
        u = self.unif()#on next line, mean is like 1/lambda, where lambda is rate
        r = - (self.mean)*ln(1.-u) #simply use u, as you normally would
        return(r)
    
    def expon2(self): #xmean is the mean; result is not really the complement
                      #of expon, so we'll just call it expon2()
        u = self.c_unif()
        r = - (self.mean)*ln(1.-u) #this complement of u
        return(r)
        


In [5]:
def main():

    arriv = Expon(2.0)
    serv  = Expon(1.0)

    arriv.setseed(1234567)
    serv.setseed(5171861)
    

#check that we get the correct u and (1-u) pairs. In the uniform case
#its easy to check because they must sum to 1
    
    for j in range(10):
        u = arriv.unif()
        arriv.restore_seed()
        u_c = arriv.c_unif()   #c_ refers to the "complement"
        xsum = u + u_c
        print(u,u_c,xsum)

#when you do the same with exponential variates, of course they will not sum
#to 1. In every pair of exponential numbers, the first number will be
# e = - xmean*ln(u),     [calling expon()]
#and the second number of the pair will be
# e2 = - xmean*ln(1-u)    [calling expon2]

    print("")

    for j in range(10):
        e = arriv.expon()  #generate an expo variate using u
        arriv.restore_seed() #want to use (1-u), **the same u as before**
        e2 = arriv.expon2() #generate the other expo variate using (1-u)
        print(e,e2)
        
        
#Remember, you will use TWO random number streams, one for all the
#interarrival times and the other for all the service times.

#You will make, say, 50 PAIRS of simulation runs.

#In the first run of each pair, say you set the seed for interarrival variates
#as "123", and for end-of-service variates as "456". Make a normal simulation
#run, trying to avoid the transient part etc, and get enough data to get some
#mean, say Y1. Every time you made a call for variates, you used "expon" with
#appropriate mean for interarrivals and end of services.

#In the second run of each pair, you will use the same seeds "123" and "456"
#as in the first run of the pair, but now you will call expon2() in every case
#instead of expon(). Now you get Y2.

#Thus whenever "u" was used in the first of the pair of runs, you must ensure
#that "(1-u)" is used in rhe second of the pair of runs.

#Since u and (1-u) have negative covariance, e and e2 will also have negative
#covariance.

#The first pair of runs will give you one observation X1 = (Y1 + Y2)/2 where
#X1 will have a smaller variance than both Y1 and Y2 because:

#Var(Y1+Y2) = Var(Y1) + Var(Y2) + 2Cov(Y1,Y2) where Cov(Y1,Y2) < 0/. This is
#trick we do.

#Var(X1) = Var((Y1 + Y2)/2) = 1/4 [Var{Y1) + Var(Y2) + 2Cov(Y1,Y2)]

#where the negative Cov(Y1,Y2) makes Var(X1) smaller than if your two runs
#in the pair were independent (which is how you would ordinarily make runs).


#NOTE: **Just make sure** that in every case where you use u, and an expon variate
#based on u in the first run of each pair of runs, you use the complement
#(1-u) and an expon variate based on (1-u) for the second run of each pair.

#Thus, if customer 5 experiences a "long" service time at some queue in
#the first run of each pair, this same customer 5 will experience a "short"
#service time at the same queue in the second run of each pair. Intuitively,
#you are compensating, or balancing "long" and "short" to reduce variation,
#even though you are still working with random times from some distribution.


        
        
    
       
        
        


In [6]:
main()

0.5543483055884251 0.44565169441157493 1.0
0.8737043686335895 0.12629563136641053 1.0
0.05902741214941041 0.9409725878505896 1.0
0.8753210354735639 0.12467896452643612 1.0
0.6738737470709896 0.32612625292901043 1.0
0.5167127693277551 0.4832872306722449 1.0
0.4888492160815838 0.5111507839184162 1.0
0.8107153645020603 0.18928463549793972 1.0
0.7285035549119142 0.2714964450880858 1.0
0.030116200333140983 0.969883799666859 1.0

0.4611471725301808 3.1605156416339413
0.020452946723534943 9.175768745053114
0.738311930766548 2.350884597966361
0.67871905397349 2.4911620105347216
1.6464511365459173 1.1561161658867936
1.3536856341940322 1.419443577082899
0.85818510770369 2.105937939024979
1.9300455445434141 0.9592177008323063
1.996219975445898 0.9195534702869073
0.48924258651121566 3.05572515840478
